# 1 ENTROPY

- Measure of uncertainty of a random variable $X$. The higher the entropy, the more the uncertainty i.e. more privacy and vice-versa.
- Discrete Entropy applies to Discrete Random Variables whilst Differential Entropy applies to  Continuous Random Variables.



## 1.1 Discrete Entropy (Discrete Random Variables)
- Let $X$ be a discrete random variable with alphabet $X$ and probability mass function $p(x) = Pr({X = x})$, $x \epsilon X$.
- The entropy $H(X)$ of a discrete random variable $X$ is defined by
$$H(X) = -\sum_{x \epsilon X}^n p(x) log p(x)$$

- NB: The log is to the base 2 and entropy is expressed in bits.
- $H(X) \geqslant 0$
- Conditioning reduces entropy (see section on conditional entropy) i.e. $H(X|Y) \leqslant H(X)$


In [11]:
# For example, we will show that the entropy of a fair coin toss is 1 bit.
'''
Let X be the outcome of a coin toss - {H,T}
P(X) = 0.5 for each outcome since it is a fair coin

----|----|----|
X   |  H | T  |
----|----|----|
P(x)| 0.5|0.5 |
----|----|----|

'''
import math

#calculating discrete entropy from first principles (alternative would be to use
#scipy library
LOG_BASE_2 = 2
OUTCOMES = ['H', 'T']
DISTRIBUTION = [0.5, 0.5]
entropy_H_X = -( sum( (i * math.log(i, LOG_BASE_2)) for i in DISTRIBUTION ) )
print('Entropy of fair coin toss is %s bit' % entropy_H_X)

#manual calculation
entropy_H_X_test = -( (0.5 * math.log(0.5, 2)) + (0.5 * math.log(0.5, 2)) )
print('Manual check of Entropy of fair coin toss is %s bit' % entropy_H_X_test)

Entropy of fair coin toss is 1.0 bit
Manual check of Entropy of fair coin toss is 1.0 bit


### 1.1.1 Conditional Entropy and Mutual Information  (Discrete Random Variables)
- Conditional entropy is the entropy of a random variable that is conditional on the knowledge of another.
- Conditioning reduces entropy i.e. $H(X|Y) \leqslant H(X)$
- Reduction in uncertainity due to another random variable is called Mutual Information(which is denoted as $I$)
- For 2 random variables, $X$ & $Y$:
$$ I(X;Y) = H(X) - H(X|Y) = \sum_{x, y}^n p(x,y) log \frac{p(x,y)}{p(x)p(y)} $$

-Mutual information $I(X;Y)$ is a measure of dependence between 2 random variables, is symmetric, always non-negative and is zero if $X$ & $Y$ are independent. In the case of privacy applications, we want mutual information to be as low or close to zero as possible.


***

## 1.2 Differential Entropy (Continuous Random Variables)
- Differential entropy differs from normal or absolute entropy in that the random variable need not be discrete.
- Interpretation of $h(X)$ is the measure for amount of information we do not have about X
- Let X be a continuous random variable with cumulative distribution function $F(x) = Pr(X \leqslant x)$ and probability density function $f(x)$ i.e. the PDF for $X$ is the first derivative of $F(x)$ which can be written as $F^\prime(x)$.
- The differential entropy $h(X)$ of a continuous random variable $X$ with density $f(x)$ is defined as 
$$h(X) = -\int_S f(x) log f(x) \delta x$$
where $S$ is the support set of the random variable (i.e. where $f(x) > 0)$

- Example (Uniform distribution) - Consider a continuous random variable, $X$, distributed uniformly from $0$ to $a$, with $a$ > 0 and density  $f(x)$ is $1/a$ from 0 to $a$ and 0 elsewhere. Then its differential entropy of $X$ is
$$h(X) = -\int_0^a \frac{1}{a} log \frac{1}{a} \delta x$$

- Differential entropy can be negative


### 1.2.1 Joint Differential Entropy
- The differential entropy of a set $X_{1}, X_{2},...X_{n}$ of continuous random variables with density $f(x_{1}, x_{2},...,x_{n})$ is
$$h(X_{1}, X_{2},...X_{n}) = -\int f(x^n) log f(x^n) \delta x^n $$

### 1.2.2 Conditional Differential Entropy
- The differential entropy of a set $X, Y$ of continuous random variables with a joint density function $f(x, y)$ is
$$h(X|Y) = -\int f(x,y) log f(x|y) \delta x \delta y $$

which can also be written as $h(X|Y) = h(X,Y) - h(Y)$ since generally bayes suggests that $f(x|y) = \frac{f(x,y)}{f(y)}$

### 1.2.3 Relative Entropy and Mutual Information
- The relative entropy (Kullback-Leibler divergence) between two densities $f$ and $g$ measures the difference between two probbility distributions and is given by
$$ D(f||g) = \int f log \frac{f}{g}$$
- The main interpretation of relative entropy is that the information we gained about $X$, if we originally thought $X ∼ g$ and now we learned X ∼ $p$ i.e. Expressed in the language of Bayesian inference, $D(f||g)$ is a measure of the information gained by revising one's beliefs from the prior probability distribution $g$ to the posterior probability distribution $p$.  In order to find a distribution $g$ that is closest to $f$, we can minimize KL divergence and compute an information projection.

- The mutual information $I(X;Y)$ between two continuous random variables with join density $f(x,y)$ is given by
$$ I(X;Y) = \int f(x,y) log \frac{f(x,y)}{f(x)f(y)} \delta x \delta y $$

In [17]:
'''
Estimating differential entropy
Estimate entropy exercies, for each, for each, 
estimate the entropy based on samples of 4 different sizes i.e. N: 10,100,1000,10000.
1. Uniform distribution on the interval [0, 1].
2. Uniform distribution on the interval [0, 8].
3. Uniform distribution on the interval [0, 0.5].
4. Gaussian distribution with mean 0 and standard deviation 1.
5. Gaussian distribution with mean 0 and standard deviation 100.
6. Exponential distribution with mean 1.
7. Exponential distribution with mean 100.

Parametric statistics are based on assumptions about the distribution of population from 
which the sample was taken. Nonparametric statistics are not based on assumptions, that is, 
the data can be collected from a sample that does not follow a specific distribution.
'''

SAMPLE_SIZES = [10,100,1000,10000]

#list to store monte carlo simulation entropies
entropy_monte_carlo = []


#create distribution
xs = []

#create standard deviation for distribution 
sigma = 1

def samplingEntropyEst(xs, N, sigma):
    '''
    Since differential entropy is an expectation (of the negative log probability density), 
    we can estimate it by sampling from a distribution and forming an empirical average of the 
    negative log probability density. This empirical average will converge to the 
    true differential entropy by the law of large numbers.

    Params:
    xs is the sample points in the distribution
    N is the number of times to sample in the Monte Carlo procedure
    sigma is the standard deviation of the sample - need to find a way of generating an 
    optimal sigma
    '''
    pass

for N in SAMPLE_SIZES:
    entropy_monte_carlo.append(samplingEntropyEst(xs, N, sigma))

def mspacingEntropyEst(xs):
    '''
    m−spacings estimate of entropy.
    One way of choosing m  is to use the nearest integer value to the square root of N, the
    number of sample points.

    Params:
    xs is the sample points in the distribution
    '''
    pass

entropy2 = mspacingEntropyEst(xs)